In [ ]:
import json
from src.cu_estimator import CuEstimator
from src.recipe_optimizer import RecipeOptimizer
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np

### Read the input datasets

In [ ]:
def load_df(path: str):
    with open(path) as f:
        data = json.load(f)
    df = json_normalize(data)
    df = df.rename(columns=lambda col: col.rpartition('.')[-1])
    return df

In [ ]:
df_prod = load_df('../data/1/previous_heats_with_properties.json')
df_prod.head()

In [ ]:
with open('../data/1/constraints.json') as f:
    data = json.load(f)['scrap_type_constraints_per_heat']
df_cons = pd.DataFrame(data)
df_cons = df_cons.replace('bushlings', 'bushling')
df_cons.head()

In [ ]:
df_orders = load_df('../data/1/scrap_orders.json')
df_orders['price_per_kg'] = df_orders['price_per_ton'] / 1000
prices = df_orders.groupby('scrap_type')['price_per_kg'].mean()

prices

---

### Build a system of linear equations and solve it with simplex method.
Current constrains:
 * Cost function = C(x) -> min
 * Mass balance = M(x) = Heat_Weight
 * Cu amount = Cu(x) <= Cu_Percent
 * Additional mass constrains from the constraints.json

Formulas:
 * Cu(x) = bush_mass * cu_amount[bash] + ... + skulls_mass * cu_amount[skulls]
 * M(x) = bush_mass + ... + skulls_mass = Heat_Weight
 * C(x) = bush_mass * price[bash] + ... + skulls_mass * price[skulls]

In [ ]:
cu_estimator = CuEstimator()
cu_estimator.fit(df_prod)

In [ ]:
recipe_optimizer = RecipeOptimizer(cu_estimator, df_cons, prices)

In [ ]:
for _, row in df_prod.iterrows():
    print(recipe_optimizer.optimize(row).x)

In [ ]:
# TODO: add scrap on hand!